In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv("../Data/Biometric Data Analysis/train.csv").fillna(0)

In [3]:
data.rho.unique()
data.drop("id", axis=1, inplace=True)

In [7]:
# MaxMin Scaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(data.iloc[:,:-4])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [9]:
# group by rho

data_rho_25 = data[data['rho'] == 25]
data_rho_20 = data[data['rho'] == 20]
data_rho_15 = data[data['rho'] == 15]
data_rho_10 = data[data['rho'] == 10]

### KNN

In [18]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [19]:
def XY(df):
    X = df.iloc[:,1:-4]
    y_hhb = df.iloc[:,-4]
    y_hhbo2 = df.iloc[:,-3]
    y_ca = df.iloc[:,-2]
    y_na = df.iloc[:,-1]
    
    return X, y_hhb, y_hhbo2, y_ca, y_na

In [31]:
def XG_Boost(df, num):
    X, y_hhb, y_hhbo2, y_ca, y_na = XY(df)
    
    # Split data
    X1_train, X1_test, y1_train, y1_test = train_test_split(X, y_hhb, test_size=0.3, shuffle=101)
    X2_train, X2_test, y2_train, y2_test = train_test_split(X, y_hhbo2, test_size=0.3, shuffle=122)
    X3_train, X3_test, y3_train, y3_test = train_test_split(X, y_ca, test_size=0.3, shuffle=157)
    X4_train, X4_test, y4_train, y4_test = train_test_split(X, y_na, test_size=0.3, shuffle=1932)
    
    model = KNeighborsRegressor(n_neighbors=37)
    
    neigh1 = model.fit(X1_train, y1_train)
    neigh2 = model.fit(X2_train, y2_train)
    neigh3 = model.fit(X3_train, y3_train)
    neigh4 = model.fit(X4_train, y4_train)
    
    preds_hhb = neigh1.predict(X1_test)
    preds_hbo2 = neigh2.predict(X2_test)
    preds_ca = neigh3.predict(X3_test)
    preds_na = neigh4.predict(X4_test)
    
    rmse_hhb = np.sqrt(mean_squared_error(y1_test, preds_hhb))
    rmse_hbo2 = np.sqrt(mean_squared_error(y2_test, preds_hbo2))
    rmse_ca = np.sqrt(mean_squared_error(y3_test, preds_ca))
    rmse_na = np.sqrt(mean_squared_error(y4_test, preds_na))

    print(num, " mm")
    print("RMSE - hbb  : %f" % (rmse_hhb))
    print("RMSE - hbo2 : %f" % (rmse_hbo2))
    print("RMSE - ca   : %f" % (rmse_ca))
    print("RMSE - na   : %f" % (rmse_na))
    
    
    return neigh1, neigh2, neigh3, neigh4

In [32]:
# data_rho_25, data_rho_20, data_rho_15, data_rho_10

hbb_25, hbo2_25, ca_25, na_25 = XG_Boost(data_rho_25, 25)
hbb_20, hbo2_20, ca_20, na_20 = XG_Boost(data_rho_20, 20)
hbb_15, hbo2_15, ca_15, na_15 = XG_Boost(data_rho_15, 15)
hbb_10, hbo2_10, ca_10, na_10 = XG_Boost(data_rho_10, 10)

25  mm
RMSE - hbb  : 5.552790
RMSE - hbo2 : 1.369242
RMSE - ca   : 6.494577
RMSE - na   : 1.890832
20  mm
RMSE - hbb  : 5.774719
RMSE - hbo2 : 1.472524
RMSE - ca   : 6.734552
RMSE - na   : 1.928604
15  mm
RMSE - hbb  : 5.849289
RMSE - hbo2 : 1.397271
RMSE - ca   : 6.597067
RMSE - na   : 1.988861
10  mm
RMSE - hbb  : 5.860116
RMSE - hbo2 : 1.452842
RMSE - ca   : 6.874648
RMSE - na   : 1.842478
